In [1]:
import pandas
import ib_insync as ibi

from datetime import datetime

global EPSILON
EPSILON = 1.e-10

In [2]:
!python --version

Python 3.7.6


In [3]:
class TimeDouble:
    """
    Basic timestamped data class.
    """
    def __init__(self,
                data_name: str,
                timestamp: datetime,
                value: float)-> None:
        """
        Initialize time double.
        
        Parameters
        ----------
        data_name: str
            Name of the data represented.
        timestamp: datetime
            Time of the realization of the data point.
        value: float
            Numerical value of the data.
        """
        self._data_name = data_name
        self._timestamp = timestamp
        self._value = value
    def get_name(self)-> str:
        return self._data_name
    def get_time(self)-> datetime:
        return self._timestamp
    def get_value(self)-> float:
        return self._value

In [4]:
from collections import deque
from datetime import timedelta
from abc import ABC, abstractmethod
from typing import List, Deque
from eventkit import Event

class SignalBase(ABC):
    """
    Base class for signal.
    """
    def __init__(self,
                input_data_array: List[str],
                warmup_length: int,
                signal_name: str)-> None:
        """
        Initialize signal base.
        
        Parameters
        ----------
        input_data_array: List[str]
            List of event subscription that is required for signal calculation.
        warmup_length: int
            Number of data points to 'burn'
        signal_name: str
            Name of the signal.
        """        
        self._input_data_array = input_data_array
        self._signal_name = signal_name
        self._warmup_length = warmup_length
        self._initialize_data_time_storage(input_data_array)
    
    def _initialize_data_time_storage(self, input_data_array: List[str])-> None:
        """
        Initialize storage.
        
        Parameters
        ----------
        input_data_array: List[str]
            List of event subscription that is required for signal calculation.
        """
        data_storage = {}
        time_storage = {}
        for input_name_ in input_data_array:
            data_storage[input_name_] = deque([])
            time_storage[input_name_] = deque([])
        self._data_storage = data_storage
        self._time_storage = time_storage
    
    def check_all_received(self, data_name: str)-> bool:
        output = True
        num_data = len(self._input_data_array)
        if num_data == 1:
            return output # If there is only one incoming data stream, no need to check
        for i in range(num_data):
            event_name_i = self._input_data_array[i]
            if len(self.get_time_by_name(event_name_i)) == 0:
                return False
            
            time_diff = self.get_time_by_name(data_name)[-1] - self.get_time_by_name(event_name_i)[-1]
            if time_diff > timedelta(microseconds = 1):
                return False
        return output
    
    def update_data(self, new_data: TimeDouble)-> None:
        """
        Update data storage.
        
        Parameters
        ----------
        new_data: TimeDouble
            Incoming new data.
        """
        # Extend storage
        self._data_storage[new_data.get_name()].append(new_data.get_value())
        self._time_storage[new_data.get_name()].append(new_data.get_time())
        # Remove oldest if warming up is complete
        if len(self._data_storage[new_data.get_name()]) > self._warmup_length:
            self._data_storage[new_data.get_name()].popleft()
            self._time_storage[new_data.get_name()].popleft()
    
    def get_data_by_name(self, data_name: str)-> Deque:
        return self._data_storage[data_name]
    
    def get_time_by_name(self, data_name: str)-> Deque:
        return self._time_storage[data_name]
    
    def get_warmup_length(self)-> int:
        return self._warmup_length
    
    def get_signal_name(self)-> str:
        return self._signal_name
    
    @abstractmethod
    def calculate_signal(self)-> float:
        """
        Virtual method to be implemented by the derived class.
        """
        pass

In [5]:
class MASignal(SignalBase):
    """
    Moving average signal.
    """
    def __init__(self,
                input_data_array: List[Event],
                warmup_length: int,
                signal_name: str)-> None:
        super().__init__(input_data_array, warmup_length, signal_name)
    def calculate_signal(self)-> float:
        """
        Compute the moving average.
        """
        prices = self.get_data_by_name(self._input_data_array[0])
        return sum(prices) / len(prices)
    

In [6]:
class WMomSignal(SignalBase):
    """
    Weighted momentum signal.
    """
    def __init__(self,
            input_data_array: List[Event],
            warmup_length: int,
            signal_name: str)-> None:
        super().__init__(input_data_array, warmup_length, signal_name)
        # Constants
        self._trading_days_in_month = 21
        self._normalization_factor = 4.0
        self._pivot_months = [1, 3, 6, 12]
    def calculate_signal(self)-> float:
        """
        Compute the weighted momentum.
        """
        prices = self.get_data_by_name(self._input_data_array[0])
        weighted_momentum = 0.0
        for _month in self._pivot_months:
            p0 = prices[-1]
            pt = prices[len(prices) - (_month * _trading_days_in_month) - 1]
            weighted_momentum += (12.0 / _month) * (p0 / pt - 1.0) / self._normalization_factor
        return weighted_momentum

In [84]:
from typing import Dict
import math

class StrategyBase(ABC):
    """
    Base class for strategy.
    """
    CASH_ACCOUNT_NAME = 'cash_account'
    
    def __init__(self,
                input_signal_array: List[str],
                trade_combo_array: List[str],
                warmup_length: int,
                initial_capital: float = 100.0)-> None:
        """
        Initialize strategy base.
        
        Parameters
        ----------
        input_signal_array: List[str]
            List of event subscription that is required for trade decision.
        trade_combo_array: List[str]
            List of event subscription that is required for ordering.   
        warmup_length: int
            Number of data points to 'burn'
        initial_capital: float
            Initial capital.
        """
        self._warmup_length = warmup_length
        self._mtm = initial_capital
        self._strategy_active = True # A strategy is active by default
        self._initialize_signal_time_storage(input_signal_array)
        self._initialize_combo_time_storage(trade_combo_array)
        self._input_signal_array = input_signal_array
        self._trade_combo_array = trade_combo_array
        
        position_temp = {sec_name: 0.0 for sec_name in trade_combo_array}
        # Remember to add cash account
        position_temp[self.CASH_ACCOUNT_NAME] = initial_capital
        self._combo_positions = position_temp
        
        self._combo_order = {sec_name: 0.0 for sec_name in trade_combo_array}
        self._average_entry_price = {sec_name: 0.0 for sec_name in trade_combo_array}
        self._mtm_price = {sec_name: 0.0 for sec_name in trade_combo_array}
        
        self._mtm_history = [initial_capital]
    
    def _initialize_signal_time_storage(self, input_signal_array: List[str])-> None:
        """
        Initialize storage.
        
        Parameters
        ----------
        input_signal_array: List[str]
            List of signal subscription that is required for ordering.
        """
        signal_storage = {}
        signal_time_storage = {}
        
        for input_name_ in input_signal_array:
            signal_storage[input_name_] = deque([])
            signal_time_storage[input_name_] = deque([])
        self._signal_storage = signal_storage
        self._signal_time_storage = signal_time_storage
        
    def _initialize_combo_time_storage(self, trade_combo_array: List[str])-> None:
        """
        Initialize storage.
        
        Parameters
        ----------
        trade_combo_array: List[str]
            List of constituents in the combo.
        """
        combo_time_storage = {}
        
        for input_name_ in trade_combo_array:
            combo_time_storage[input_name_] = deque([])
        self._combo_time_storage = combo_time_storage
        
    def get_strategy_active(self)-> bool:
        return self._strategy_active
        
    def set_strategy_active(self, activity: bool)-> None:
        self._strategy_active = activity
    
    def check_all_signals_received(self, signal_name: str)-> bool:
        output = True
        num_signal = len(self._input_signal_array)
        if num_signal == 1:
            return output # If there is only one incoming signal stream, no need to check
        for i in range(num_signal):
            event_name_i = self._input_signal_array[i]
            if len(self.get_signal_time_by_name(event_name_i)) == 0:
                return False
            
            time_diff = self.get_signal_time_by_name(signal_name)[-1] - self.get_signal_time_by_name(event_name_i)[-1]
            if time_diff > timedelta(microseconds = 1):
                return False
        return output
    
    def update_data(self, new_data: TimeDouble)-> None:
        """
        Update signal storage.
        
        Parameters
        ----------
        new_data: TimeDouble
            Incoming new data.
        """
        # Extend storage
        self._signal_storage[new_data.get_name()].append(new_data.get_value())
        self._signal_time_storage[new_data.get_name()].append(new_data.get_time())
        # Remove oldest if warming up is complete
        if len(self._signal_storage[new_data.get_name()]) > self._warmup_length:
            self._signal_storage[new_data.get_name()].popleft()
            self._signal_time_storage[new_data.get_name()].popleft()
    
    def get_signal_time_by_name(self, signal_name: str)-> Deque:
        return self._signal_time_storage[signal_name]
    
    def get_signal_by_name(self, signal_name: str)-> Deque:
        return self._signal_storage[signal_name]
    
    def check_all_combo_data_received(self, data_name: str)-> bool:
        output = True
        num_data = len(self._trade_combo_array)
        if num_data == 1:
            return output # If there is only one incoming data stream, no need to check
        for i in range(num_data):
            event_name_i = self._trade_combo_array[i]
            if len(self.get_combo_time_by_name(event_name_i)) == 0:
                return False
            
            time_diff = self.get_combo_time_by_name(data_name)[-1] - self.get_combo_time_by_name(event_name_i)[-1]
            if time_diff > timedelta(microseconds = 1):
                return False
        return output
    
    def undate_mtm(self, new_data: TimeDouble)-> None:
        """
        Update data storage used for MTM calculation.
        
        Parameters
        ----------
        new_data: TimeDouble
            Incoming new data.
        """
        self._mtm_price[new_data.get_name()] = new_data.get_value()
        self.get_combo_time_by_name(new_data.get_name()).append(new_data.get_time())
        if len(self.get_combo_time_by_name(new_data.get_name())) > 1:
            self.get_combo_time_by_name(new_data.get_name()).popleft()
    
    def get_combo_time_by_name(self, data_name: str)-> Deque:
        return self._combo_time_storage[data_name]
    
    def get_signal_containing_string(self,
                                     input_string: str)-> Deque:
        """
        Retrieve the signals that contain a certain string in the name.
        
        Parameters
        ----------
        input_string: str
            String that is supposed to be part of the name.
            
        Returns
        -------
            Deque
                The signal data that is stored.
        """
        for key, value in self._signal_storage.items():
            if input_string in key:
                return value
        # If not found
        raise ValueError('StrategyBase.get_signal_containing_string: Signal not found.')
    
    def send_order(self)-> None:
        """
        Send pending orders and use latest price to calculate average cost.
        """
        for key, value in self._combo_order.items():
            # Adjust cash account
            self._combo_positions[self.CASH_ACCOUNT_NAME] -= value * self._mtm_price[key]
            # Calculate average cost
            self._average_entry_price[key] = self._calculate_new_average_entry_price(self._average_entry_price[key],
                                                                                    self._mtm_price[key],
                                                                                    self._combo_positions[key],
                                                                                    value)
            # Modify current position
            self._combo_positions[key] += value
            # Reset pending order vector
            # TBD: Check to make sure this is actually updating the class member field
            value = 0.0
    
    def _calculate_new_average_entry_price(self,
                                  old_average_entry_price: float,
                                  new_entry_price: float,
                                  existing_position: float,
                                  new_pending_order: float)-> float:
        """
        Calculate new average entry price.
        
        Parameters
        ----------
        old_average_entry_price: float
            Existing average entry price.
        new_entry_price: float
            New entry price.
        existing_position: float
            Existing position.
        new_pending_order: float
            New pending order.
            
        Returns
        -------
            float
                The new average entry price.
        """
        if math.copysign(1, existing_position) != math.copysign(1, existing_position + new_pending_order):
            # If going from net long to net short or vice versa
            return new_entry_price
        elif abs(existing_position + new_pending_order) > 0.0:
            return (existing_position * old_average_entry_price + new_pending_order * new_entry_price) / (existing_position + new_pending_order)
        else:
            return new_entry_price
    
    def calculate_mtm(self)-> None:
        """
        Mark to market.
        """
        mtm = 0.0
        for key, value in self._mtm_price.items():
            mtm += value * self._combo_positions[key]
        mtm += self._combo_positions[self.CASH_ACCOUNT_NAME]
        self._update_mtm_history(mtm)
    
    def _update_mtm_history(self, new_mtm: float)-> None:
        """
        Update MTM history.
        
        Parameters
        ----------
        new_mtm: float
            Latest MTM.
        """
        self._mtm_history.append(new_mtm)
    
    def has_pending_order(self)-> bool:
        """
        Check if there is any pending order.
        """
        for key, value in self._combo_order.items():
            if abs(value) > 0.0:
                return True
        return False
    
    def has_pending_order(self, contract_name: str)-> bool:
        """
        Check if a specific contract has any pending order.
        """
        if abs(self._combo_order[contract_name]) > 0.0:
            return True
        else:
            return False
    
    def is_currently_long(self, contract_name: str)-> bool:
        return self._combo_positions[contract_name] > 0.0

    def is_currently_short(self, contract_name: str)-> bool:
        return self._combo_positions[contract_name] <= 0.0

    def is_currently_flat(self, contract_name: str)-> bool:
        return abs(self._combo_positions[contract_name]) <= EPSILON
    
    def get_warmup_length(self)-> int:
        return self._warmup_length
    
    @abstractmethod
    def make_order_decision(self)-> Dict[str, float]:
        """
        Virtual method to be implemented by the derived class.
        """
        pass

In [85]:
class MACrossingStrategy(StrategyBase):
    """
    MA Crossing Strategy
    """
    def __init__(self,
                input_signal_array: List[str],
                trade_combo_array: List[str],
                warmup_length: int)-> None:
        """
        Initialize MA Crossing Strategy.
        
        Parameters
        ----------
        input_signal_array: List[str]
            List of event subscription that is required for trade decision.
        trade_combo_array: List[str]
            List of event subscription that is required for ordering.   
        warmup_length: int
            Number of data points to 'burn'
        """
        super().__init__(input_signal_array, trade_combo_array, warmup_length)
        self._long_signal_name, self._short_signal_name = self._deduce_signal_name(input_signal_array)
        
    def _deduce_signal_name(self, input_signal_array: List[str])-> (str, str):
        """
        Deduce the name of the signal with longer (shorter) average window.
        
        Parameters
        ----------
        input_signal_array: List[str]
            List of event subscription that is required for trade decision.
            
        Returns
        -------
            str, str
                The long and short window signal names respectively.
        """
        name_1 = input_signal_array[0]
        name_2 = input_signal_array[1]
        delimiter = '_'
        substr_1 = name_1.split(delimiter)[-1]
        substr_2 = name_2.split(delimiter)[-1]
        if not (isinstance(int(substr_1), int) and isinstance(int(substr_2), int)):
            raise TypeError('MACrossingStrategy._deduce_signal_name: Signal name in wrong format.')
        if int(substr_1) > int(substr_2):
            return name_1, name_2
        else:
            return name_2, name_1
        
    def make_order_decision(self)-> Dict[str, float]:
        for key, value in self._combo_positions.items():
            if key != self.CASH_ACCOUNT_NAME:
                contract_name = key
        current_price = self._mtm_price[contract_name]
        if not self.has_pending_order(contract_name):
            if self._signal_storage[self._long_signal_name][-1] >= self._signal_storage[self._short_signal_name][-1] and (self.is_currently_long(contract_name) or self.is_currently_flat(contract_name)):
                return {contract_name: -self._mtm_history[-1] / current_price - self._combo_positions[contract_name]}
            elif self._signal_storage[self._long_signal_name][-1] < self._signal_storage[self._short_signal_name][-1] and (self.is_currently_short(contract_name) or self.is_currently_flat(contract_name)):
                return {contract_name: self._mtm_history[-1] / current_price - self._combo_positions[contract_name]}
            else:
                return {contract_name: 0.0}
        else:
            return {contract_name: 0.0}

In [68]:
class DataSlot:
    """
    Class for data slot.
    """
    def __init__(self,
                data_name: str,
                parent_signals: List[SignalBase])-> None:
        """
        Initialize data slot.

        Parameters
        ----------
        data_name: str
            Name of the input data.
        parent_signals: List[SignalBase]
            Signals that listens to this data.
        """
        self._data_name = data_name
        self._parent_signals = parent_signals
        signal_events = {}
        for _parent in self._parent_signals:
            signal_events[_parent.get_signal_name()] = Event(name = _parent.get_signal_name())
        self._signal_events = signal_events

    def get_signal_events(self)-> Dict[str, Event]:
        return self._signal_events
        
    def on_event(self, new_data: TimeDouble)-> None:
        """
        Perform the action upon getting an event.

        When there is an event (arrival of data) we want to
        - Update data storage
        - Calculate signal if warming up is complete
        
        Parameters
        ----------
        new_data: TimeDouble
            Incoming new data.
        """
        for _parent in self._parent_signals:
            # 1. Update data storage
            _parent.update_data(new_data)

            # 2. If warming up is complete and all data arrived, calculate and publish signal
            if len(_parent.get_data_by_name(new_data.get_name())) == _parent.get_warmup_length() and _parent.check_all_received(new_data.get_name()):
                signal = _parent.calculate_signal()
                ### DEBUG
                print(signal)
                latest_timestamp = _parent.get_time_by_name(new_data.get_name())[-1]
                self._signal_events[_parent.get_signal_name()].emit(TimeDouble(_parent.get_signal_name(), latest_timestamp, signal))

In [41]:
class SlotBase:
    """
    Base class for slot, to be derived for Strategy.
    """
    def __init__(self,
                data_name: str,
                parent_strategies: List[StrategyBase])-> None:
        """
        Initialize base slot.

        Parameters
        ----------
        data_name: str
            Name of the input data.
        parent_strategies: List[StrategyBase]
            Strategies that listens to this data.
        """
        self._data_name = data_name
        self._parent_strategies = parent_strategies

    @abstractmethod
    def on_event(self, new_data: TimeDouble)-> None:
        """
        Perform the action upon getting an event.
        """
        pass
    
class SignalSlot(SlotBase):
    """
    Class for signal slot.
    """
    def __init__(self,
                data_name: str,
                parent_strategies: List[StrategyBase])-> None:
        super().__init__(data_name, parent_strategies)
        
        
    def on_event(self, new_signal: TimeDouble)-> None:
        """
        Perform the action upon getting an event.

        When there is an event (arrival of signal) we want to
        - Update signal storage
        - Make trade decision if warming up is complete
        
        Parameters
        ----------
        new_signal: TimeDouble
            Incoming new signal.
        """
        for _parent in self._parent_strategies:
            # 1. Update signal storage
            _parent.update_data(new_signal)

            # 2. If warming up is complete and all data arrived, make trade decision
            if len(_parent.get_signal_by_name(new_signal.get_name())) == _parent.get_warmup_length() and _parent.check_all_signals_received(new_signal.get_name()):
                if _parent.get_strategy_active():
                    # DEBUG
                    print(_parent.make_order_decision())
                    _parent._combo_order = _parent.make_order_decision()
                    
class MTMPriceSlot(SlotBase):
    """
    Class for marking to market price slot.
    """
    def __init__(self,
                data_name: str,
                parent_strategies: List[StrategyBase])-> None:
        super().__init__(data_name, parent_strategies)
        
    def on_event(self, new_data: TimeDouble)-> None:
        """
        Perform the action upon getting an event.

        When there is an event (arrival of contract prices) we want to
        - Update mtm price storage
        - Send order
        - Mark to market
        
        Parameters
        ----------
        new_data: TimeDouble
            Incoming new data.
        """
        for _parent in self._parent_strategies:
            # 1. Update mtm price storage
            _parent.undate_mtm(new_data)
            if _parent.check_all_combo_data_received(new_data.get_name()):
                # 2. Send order
                _parent.send_order()
                # 3. Mark to market
                _parent.calculate_mtm()

In [11]:
class EventContext:
    """
    Class for event context.
    TBD: Make it a singleton
    """
    def __init__(self, broker_handle: ibi.IB, data_handle: ibi.IB)-> None:
        """
        Initialize event context.
        
        Parameters
        ----------
        broker_handle: ibi.IB
            Broker API handle.
        data_handle: ibi.IB
            Data source handle.
        """
        self._broker_handle = broker_handle
        self._data_handle = data_handle
    def get_broker(self)-> ibi.IB:
        return self._broker_handle
    def get_data(self)-> ibi.IB:
        return self._data_handle
    
class BrokerEventRelay:
    """
    Class for broker event relay.
    TBD: Allow for different brokers (create derived classes)
    TBD: Add different relay member functions (open, high, low, close, volume)
    """
    def __init__(self,
                 listener: DataSlot,
                 data_name: str,
                 field_name: str = 'close',
                 )-> None:
        """
        Initialize broker event relay.
        
        Parameters
        ----------
        listener: DataSlot
            Listener data slot.
        data_name: str
            Name of the input data.
        field_name: str
            Data field name (open, high, low, close, volume, etc.). 
        """
        self._relay_event = Event()
        self._field_name = field_name
        self._relay_event += listener.on_event
        self._data_name = data_name
        
    def ib_live_bar(self,
                    bars: ibi.RealTimeBarList,
                    has_new_bar: bool)-> None:
        """
        Translate IB real time bar event into price update.
        
        Parameters
        ----------
        bars: ibi.RealTimeBarList
            IB RealTimeBarList.
        has_new_bar: bool
            Whether there is new bar.
        """
        if has_new_bar:
            if self._field_name == 'close':
                field = bars[-1].close
            else:
                raise TypeError('BrokerEventRelay.ib_live_bar: Unsupported data field type.')
            relay_data = TimeDouble(self._data_name, bars[-1].time, field)
            self._relay_event.emit(relay_data)

In [12]:
ibi.util.startLoop()
                
# Connection
ib = ibi.IB()
ib.connect('127.0.0.1', 4002, clientId=13)

<IB connected to 127.0.0.1:4002 clientId=13>

In [86]:
my_ec = EventContext(ib, ib)

# Subscription
my_contract = ibi.Forex('EURUSD')   
live_bars = ib.reqRealTimeBars(my_contract, 5, 'MIDPOINT', False)

# Create signals
input_data_name = 'EURUSD_close'
my_ma_signal_short = MASignal([input_data_name], 2, 'EURUSD_MA_2')
my_ma_signal_long = MASignal([input_data_name], 3, 'EURUSD_MA_3')
my_eurusd_data = DataSlot(input_data_name, [my_ma_signal_short, my_ma_signal_long])
my_ber = BrokerEventRelay(my_eurusd_data, input_data_name)

# Create strategy
my_macrossing_strategy = MACrossingStrategy(['EURUSD_MA_2', 'EURUSD_MA_3'], [input_data_name], 1)
my_ma_short_data = SignalSlot('EURUSD_MA_2', [my_macrossing_strategy])
my_ma_long_data = SignalSlot('EURUSD_MA_3', [my_macrossing_strategy])
my_eurusd_data.get_signal_events()['EURUSD_MA_2'] += my_ma_short_data.on_event
my_eurusd_data.get_signal_events()['EURUSD_MA_3'] += my_ma_long_data.on_event
my_mtm_data = MTMPriceSlot('EURUSD_close', [my_macrossing_strategy])
my_ber2 = BrokerEventRelay(my_mtm_data, input_data_name)

live_bars.updateEvent += my_ber.ib_live_bar
live_bars.updateEvent += my_ber2.ib_live_bar

# Ending
ib.sleep(40)
ib.cancelRealTimeBars(live_bars)

1.17763
1.1776274999999998
1.1776283333333333
{'EURUSD_close': -84.9163149715955}
1.177625
1.1776266666666666
{'EURUSD_close': 0.0}
1.177625
1.177625
{'EURUSD_close': 0.0}
1.1776149999999999
1.1776183333333332
{'EURUSD_close': 0.0}
1.177605
1.1776116666666665
{'EURUSD_close': 0.0}
1.177665
1.177645
{'EURUSD_close': 169.83587486332442}


In [ ]:
ib.disconnect()